In [1]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [13]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from keras.models import load_model


In [14]:
batch_size=32
num_classes=10
epochs=2
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [15]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [16]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train/=255
x_test/=255

In [17]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [18]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding='Same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='Same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [19]:
opt=keras.optimizers.rmsprop(lr=0.0001,decay=1e-6)

In [20]:
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.summary()

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x0000020AEAC12E80>>


In [21]:

history=model.fit(x_train,y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test,y_test))
#batch size here is how many nodes in hidden layer
score=model.evaluate(x_test,y_test,verbose=0)
print('Test losses',score[0])
print('Test accuracy',score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 137s 3ms/step - loss: 1.7624 - acc: 0.3613 - val_loss: 1.5420 - val_acc: 0.4397
Epoch 2/2
50000/50000 [==============================] - 133s 3ms/step - loss: 1.4677 - acc: 0.4717 - val_loss: 1.3430 - val_acc: 0.5267
Test losses 1.3430126508712767
Test accuracy 0.5267


In [22]:
model.save('cifar_cnn_5.h5')

In [23]:
import cv2
import numpy as np
from keras.models import load_model

In [24]:
img_row,img_height,img_depth=32,32,3
classifier=load_model('cifar_cnn_5.h5')
color=True
Scale=8
def draw_test(name,res,input_im,Scale,img_row,img_height):
    BLACK=[0,0,0]
    res=int(res)
    if res==0:
        pred='airplane'
    if res==1:
        pred='automobile'
    if res==2:
        pred='bird'
    if res==3:
        pred='cat'
    if res==4:
        pred='deer'
    if res==5:
        pred='dog'
    if res==6:
        pred='frog'
    if res==7:
        pred='horse'
    if res==8:
        pred='ship'
    if res==9:
        pred='truck'
    expanded_image=cv2.copyMakeBorder(input_im,0,0,0,imageL.shape[0]*2,cv2.BORDER_CONSTANT,value=BLACK)
    if color==False:#if the image is black n white then it converts it into bgr
        expanded_image=cv2.cvtColor(expanded_image,cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image,str(pred),(300,80),cv2.FONT_HERSHEY_COMPLEX_SMALL,3,(0,255,0),2)
    cv2.imshow(name,expanded_image)
    


In [25]:
for i in range(10):
    rand=np.random.randint(0,len(x_test))
    input_im=x_test[rand]
    imageL=cv2.resize(input_im,None,fx=Scale,fy=Scale,interpolation=cv2.INTER_CUBIC)#32  pixel thi but ab wo 32*8=256 hogya i.e xaxis  and yaxis *8 i.e enlarge kr diya image ko
    #every pixel gets enlarged by 8 and by intercubic method so interpolation is intercubic
    cv2.imshow('imageL',imageL)
    print(imageL.shape)
    input_im=input_im.reshape(1,img_row,img_height,img_depth)
    print(input_im.shape)
    res=str(classifier.predict_classes(input_im,1,verbose=0)[0])
    draw_test('Prediction',res,imageL,Scale,img_row,img_height)
    cv2.waitKey(0)
cv2.destroyAllWindows()

(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
(256, 256, 3)
(1, 32, 32, 3)
